Set working directory

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import logging

# Change file_path as necessary
file_path = "../Relevant data" 

os.chdir(file_path)

Load files

In [2]:
def identify_type_of_day(date):
    weekno = date.weekday()
    #Ignoring the major study breaks as this is not much importance
    #if((date.month == 7 and week_of_month(date) < 4 ) or (date.month == 12 or date.month == 1 or date.month == 2)):
    #    return 'Major_study_break'
    if weekno>4:
        return 'Weekend'
    elif is_public_holiday(date):
        return 'Public_holiday'
    else:
        return 'Normal_Weekday'

#Week of month
from math import ceil

def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """
    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

#Returns True if it's  public holiday
def is_public_holiday(dt):
    #public_holiday file for year 2017, use API/ file having multiple year's records to apply for different years
    public_holidays =  pd.read_csv('australianpublicholidays.csv', sep=',', encoding='ISO-8859-1', parse_dates=['Date'],
                 date_parser=lambda x: pd.to_datetime(x))
    """"Instead API could be used : import urllib
    url = 'https://data.gov.au/api/3/action/datastore_search?resource_id=253d63c0-af1f-4f4c-b8d5-eb9d9b1d46ab&limit=5&q=title:jones'  
    fileobj = urllib.urlopen(url)
    print fileobj.read() """
    
    #get public holidays of SA only
    public_holidays_SA = public_holidays[public_holidays['Applicable To'].str.contains("SA") | public_holidays['Applicable To'].str.contains("NAT")]
    
    #Return True if each date is public holiday in SA
    return dt in (public_holidays_SA['Date'].apply(lambda x: datetime.date(x))).values


In [3]:
#reads a csv file, adds a day type column, creates a csv file with day type column and returns a dataframe object with day type
def add_day_type_column(file, date_column):
    data_file = pd.read_csv(file, sep=',', encoding='ISO-8859-1', parse_dates=[date_column],
                 date_parser=lambda x: pd.to_datetime(x.rsplit(' ', 1)[0]))
    data_file['Type_of_day'] = data_file[date_column].apply(lambda x: identify_type_of_day(datetime.date(x)))
    file_name = file + '_with_day_type.csv'
    #uncomment next line if you want a new file to be created with day type column
    #data_file.to_csv(file_name, sep=',', encoding='utf-8')
    return data_file


In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# ### estimate campus power
# 
# The bulk of the data is for kVA values. Estimated the real power drawn from this based on the mean power factor for the data we did have. 

# In[8]:

def campus_power_estimation(bms):
    """Estimate real power form kVA value based on average pf of the 
        small set of paired values we do have
    """
    
    #bms[bms['ML_PH/Main_11kV_kW (kW)'] > 0.0]['ML_PH/Main_11kV_kW (kW)'] / bms[bms['ML_PH/Main_11kV_kW (kW)'] > 0.0]['ML_PH/Main_11kV_kVA (kVA)']

   #if logger.getEffectiveLevel() < logging.INFO: 
        #plt.scatter(bms[bms['ML_PH/Main_11kV_kW (kW)'] > 0.0]['ML_PH/Main_11kV_kW (kW)'], 
         #           bms[bms['ML_PH/Main_11kV_kW (kW)'] > 0.0]['kVA'], 
          #          alpha=0.05)
        #plt.title('Power factor scatter plot.'); 
        #plt.show()
    
    bms['kW_est'] = bms['kVA'] * 0.92
    # use original values where I have them: 
    #bms['kW_est'] = np.where(bms['kW'] == 0.0, bms['kW_est'], bms['kW'])
    return bms


In [5]:
#replace zero's with previous year values
def replace_zeros(bms,bms_lastyear):
    #replace year in timestamp
    imputed_df = bms.where((bms['kVA'] == 0) & (bms['TimeStamp'] == bms_lastyear['TimeStamp']), bms_lastyear['kVA'],axis=0)
    return imputed_df

In [6]:
#2016 demand plotline
bms_demand_2016 = add_day_type_column("bms_2016_kva.csv",'?Timestamp')
bms_demand_2016.rename(columns={'?Timestamp': 'TimeStamp','ML_PH/Main_11kV_kVA (kVA)' : 'kVA','ML_PH/PH_Ambient_Temperature (°C)':'Temp','ML_PH/PH_Ambient_Humidity (%RH)': 'Humidity'}, inplace=True)
bms_demand_2016['kVA'].plot.line()

KeyboardInterrupt: 

In [ ]:
#read data file and add type of day column to it
bms_demand_2017 = add_day_type_column('bms_2017_kva.csv','?Timestamp')
bms_demand_2017.rename(columns={'ML_PH/Main_11kV_kW (kW)': 'kW','?Timestamp': 'TimeStamp','ML_PH/Main_11kV_kVA (kVA)' : 'kVA','ML_PH/PH_Ambient_Temperature (°C)':'Temp','ML_PH/PH_Ambient_Humidity (%RH)': 'Humidity'}, inplace=True)
bms_demand_2017_est = campus_power_estimation(bms_demand_2017)
bms_demand_2017['kW_est'].plot.line()

#imputed kva's plot line (replaced from last year values)
#bms_2017_imputed = replace_zeros(bms_demand_2017, bms_demand_2016)
#bms_2017_imputed['kVA'].plot.line()

#Estimate kW using kVA if kW values not recorded
#bms_demand_2017_est = campus_power_estimation(bms_2017_imputed)
#bms_demand_2017_est['kW_est'].plot.line()


In [ ]:
#construct data sets for years 2013 to 2017
bms_demand_201314 = add_day_type_column("bms_1314_kva.csv",'?Timestamp')
bms_demand_201314.rename(columns={'?Timestamp': 'TimeStamp','ML_PH/Main_11kV_kVA (kVA)' : 'kVA','ML_PH/PH_Ambient_Temperature (°C)':'Temp','ML_PH/PH_Ambient_Humidity (%RH)': 'Humidity'}, inplace=True)
bms_demand_2015 = add_day_type_column("bms_2015_kva.csv",'?Timestamp')
bms_demand_2015.rename(columns={'?Timestamp': 'TimeStamp','ML_PH/Main_11kV_kVA (kVA)' : 'kVA','ML_PH/PH_Ambient_Temperature (°C)':'Temp','ML_PH/PH_Ambient_Humidity (%RH)': 'Humidity'}, inplace=True)

In [ ]:
bms_13to17_kva = pd.concat([bms_demand_2017,bms_demand_2016,bms_demand_2015,bms_demand_201314],sort = True)
bms_13to17_kva = bms_13to17_kva.sort_values('TimeStamp').reset_index()
bms_1317_kW_est = campus_power_estimation(bms_13to17_kva)
bms_1317_kW_est

In [ ]:
bms_1317_kW_est.to_csv("bms_13to17_kW_est.csv", sep=',', encoding='utf-8')

In [ ]:
#test code for imputation
test_2017 = bms_demand_2017
test_2016 = bms_demand_2016
#bms_demand_2017.where(bms_demand_2017['kVA'] != 0 && bms_demand_2017['TimeStamp'].apply(lambda x: datetime.date(x)) 
 #                     , bms_demand_2016['kVA'],axis=0)
test_2016_without_29 = test_2016.drop(test_2016[(test_2016['TimeStamp'] >= '2016-02-29 00:00:00') & (test_2016['TimeStamp'] < '2016-03-01 00:00:00')].index).reset_index()
test_2017.sort_index(inplace=True)
test_2016_without_29['TimeStamp'] = test_2016_without_29['TimeStamp'].apply(lambda x: x.replace(year = 2017))
df_without_zero = replace_zeros(test_2017,test_2016_without_29)

df_without_zero.kW_est.plot()

In [ ]:
# Distribution of consumption during different day type's
grouped_by_daytype = bms_1317_kW_est.groupby(['Type_of_day'])['kW_est'].mean()
grouped_by_daytype
grouped_by_daytype.plot.bar()

In [ ]:
# change scale of humidity and temperature
bms_transformed = bms_demand_2017_est[['TimeStamp','kW_est', 'Type_of_day','Humidity','Temp']].copy()
bms_transformed['Humidity'] = bms_transformed['Humidity'].apply(lambda x: x*10)
bms_transformed['Temp'] = bms_transformed['Temp'].apply(lambda x: x*10)
bms_transformed.loc[5000:7000].plot(x='TimeStamp', y=['kW_est','Humidity','Temp'] ,figsize=(20,10), grid=True  )

In [ ]:
#Check correations among attributes
corr_val = bms_1317_kW_est.corr()
print (corr_val)
import seaborn as sns
sns.heatmap(corr_val, 
        xticklabels=corr_val.columns,
        yticklabels=corr_val.columns)

In [ ]:
import math
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMAResults

In [ ]:
demand = bms_demand_2017_est['kW_est']

plt.plot(demand)
plt.show()
acf_1 =  acf(demand)[1:20]
plt.plot(acf_1)
plt.show()
test_df = pd.DataFrame([acf_1]).T
test_df.columns = ['Pandas Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
pacf_1 =  pacf(demand)[1:20]
plt.plot(pacf_1)
plt.show()
test_df = pd.DataFrame([pacf_1]).T
test_df.columns = ['Pandas Partial Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
result = ts.adfuller(demand, 1)
result

In [ ]:
demand_matrix=demand.as_matrix()
model = ARIMA(demand_matrix, order=(3,1,3))
model_fit = model.fit(disp=0)
fits = model_fit.fittedvalues
residuals = model_fit.resid
print(model_fit.summary())
#predictions = model_fit.predict(2,17600, typ = 'levels')
#predictions
#predictionsadjusted = np.exp(predictions)
#plt.plot(predictionsadjusted)
#plt.show()

#model_fit.plot_diagnostics(figsize=(15, 12))

In [ ]:
#results = ARMAResults(model,predictions)
#fittedValues = results.fittedvalues
#append 0 in first element
fits = np.concatenate([[0],fits])
bms_demand_2017_est['fits'] = bms_demand_2017_est['kW_est'] + fits
bms_demand_2017_est.loc[5000:7000].plot(x='TimeStamp', y=['kW_est','fits'] ,figsize=(20,10), grid=True  )

In [ ]:
#ARIMAX model
arimax_model=ARIMA(endog=bms_demand_2017_est['kW_est'],exog=bms_demand_2017_est[['Temp','Humidity']],order=[3,1,3])
results3=arimax_model.fit()
print(results3.summary())

In [ ]:
# Future fits
current_values = bms_demand_2017_est.fits[17500:]
forecast = model_fit.predict(start=17500, end=17600)
print(bms_demand_2017_est.fits[17500:])
print(forecast)
#bms_demand_2017_est.insert(17500:17600, 'fits', forecast)


In [ ]:
#validation data set
validation_2018 = add_day_type_column('validation_2018_data.csv','?Timestamp')
validation_2018.rename(columns={'ML_PH/Main_11kV_kW (kW)': 'kW','?Timestamp': 'TimeStamp','ML_PH/Main_11kV_kVA (kVA)' : 'kVA','ML_PH/PH_Ambient_Temperature (°C)':'Temp','ML_PH/PH_Ambient_Humidity (%RH)': 'Humidity'}, inplace=True)
validation_2018_est = campus_power_estimation(validation_2018)
validation_2018_est.to_csv("validation_2018_est.csv", sep=',', encoding='utf-8')
validation_2018_est.kW_est.plot()

In [ ]:
validation_2018_est